In [5]:
# note: same thing for REPL
# note: we use this instead of magic because `black` will otherwise fail to format
#
# Enable autoreload to automatically reload modules when they change

from IPython import get_ipython

# do this so that formatter not messed up
ipython = get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

# Import commonly used libraries
# import numpy as np
import pandas as pd

# more itertools
import more_itertools as mi

# itertools
import itertools
import collections

# tensor manipulation
# from einops import rearrange, reduce, repeat

# automatically apply jaxtyping
# %load_ext jaxtyping
# %jaxtyping.typechecker typeguard.typechecked

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os
import pathlib

import openai

MarkdownStr = str


def concatenate_md_files(root_dir: pathlib.Path) -> MarkdownStr:
    md_contents = []

    # Combine iterators for .md and .qmd files
    md_qmd_files = itertools.chain(root_dir.rglob("**/*.md"), root_dir.rglob("**/*.qmd"))

    for filepath in md_qmd_files:
        content = filepath.read_text(encoding="utf-8")
        md_contents.append(content)

    concatenated_content = "\n\n".join(md_contents)

    return concatenated_content

In [21]:
root_dir = pathlib.Path("inspect_ai/docs")

concatenated_md_files = concatenate_md_files(root_dir)

print(len(concatenated_md_files))

280256


In [15]:
# Store the concatenated markdown files in a text file so we can sanity check it
output_file_path = "concatenated_md_files.txt"

# Write the concatenated content to the file
# Using 'with' ensures the file is properly closed after writing
with open(output_file_path, "w", encoding="utf-8") as output_file:
    output_file.write(concatenated_md_files)

print(f"Concatenated markdown files have been stored in: {output_file_path}")

Concatenated markdown files have been stored in: concatenated_md_files.txt


In [ ]:
import tiktoken
import tqdm

# Create a GPT-4o tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")

# create an openai client
openai_client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [28]:
def count_tokens(text: str) -> int:
    tokens = tokenizer.encode(text)
    return len(tokens)


print(f"Number of characters: {len(concatenated_md_files)}")
print(f"Number of tokens: {count_tokens(concatenated_md_files)}")

Number of characters: 280256
Number of tokens: 63537


In [41]:
def generate_cheat_sheet_prompt(concatenated_content: MarkdownStr) -> str:

    cheat_sheet_prompt = rf"""
You are tasked with creating a comprehensive yet compact guide based on the markdown documentation of a Python library. This guide will serve as a reference for another AI model when assisting users with the library. Your goal is to distill the essential information into a concise format that captures the key functionality and usage of the library.

Here is the markdown documentation for the Python library:

<docstring>
{concatenated_content}
</docstring>

Follow these steps to create the guide:

1. Carefully read through the entire docstring to understand the structure and content of the library documentation.

2. Identify the main components of the library, such as:
   - Core classes
   - Important functions
   - Key modules
   - Significant features

3. For each main component, extract and summarize the following information:
   - Brief description (1-2 sentences)
   - Key parameters or attributes
   - Important methods (for classes)
   - Usage examples (if provided and crucial for understanding)

4. Organize the information in a hierarchical structure, starting with the most fundamental elements of the library and progressing to more specific or advanced features.

5. Use concise language and avoid redundancy. Aim to capture the essence of each component in as few words as possible while maintaining clarity.

6. If the library has a clear workflow or common usage patterns, include a brief overview of these at the beginning of the guide.

7. Omit any information related to command-line tools or installation procedures.

8. If there are deprecated features or version-specific notes, include only if they are critical for understanding the current functionality of the library.

9. Format your guide using markdown syntax for better readability. Use headers (##, ###) to separate sections and subsections.

10. Keep the total length of the guide to a few thousand tokens (approximately 2000-3000 words).

Structure your output as follows:

<guide>
## [Library Name] Quick Reference Guide

[Brief introduction to the library and its primary purpose]

### Core Components
[List and briefly describe the main classes, functions, or modules]

### Key Functionality
[Summarize the most important features and capabilities]

### Detailed Component Descriptions
[For each major component, provide a concise description, key parameters/methods, and brief usage examples if crucial]

### Common Workflows (if applicable)
[Outline typical usage patterns or workflows]

### Additional Notes
[Any critical information that doesn't fit into the above categories]
</guide>

Remember to focus on creating a guide that is both comprehensive and compact. It needs to contain enough information that another AI model would be able to assist users effectively with this library without additional reference material."""

    return cheat_sheet_prompt


def generate_cheat_sheet(client: openai.OpenAI, concatenated_content: MarkdownStr) -> str:

    # Prepare the messages for the Chat Completion API
    messages = [
        {
            "role": "system",
            "content": "You are an AI language model that generates comprehensive guides for programming libraries that are used as a reference for larger models.",
        },
        {
            "role": "user",
            "content": generate_cheat_sheet_prompt(concatenated_content),
        },
    ]

    # Call the OpenAI Chat Completion API
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=2048,
        temperature=1.0,
        # n=1,
    )

    cheat_sheet = response.choices[0].message.content.strip()

    print(f"Generated cheat sheet length (chars): {len(cheat_sheet)}")
    print(f"Generated cheat sheet length (tokens): {count_tokens(cheat_sheet)}")
    print(f"Generated cheat sheet: {cheat_sheet}")

    return cheat_sheet

In [42]:
cheat_sheet = generate_cheat_sheet(openai_client, concatenated_md_files)

Generated cheat sheet length (chars): 4491
Generated cheat sheet length (tokens): 995
Generated cheat sheet: <guide>
## Inspect Quick Reference Guide

Inspect is a Python framework for evaluating large language models (LLMs) with ease and flexibility. It provides built-in components for dataset handling, solvers, scoring mechanisms, and advanced error management.

### Core Components
- **Sample**: The main data structure for holding inputs, targets, choices, IDs, and metadata for evaluation tasks.
- **Task**: Defines an evaluation task comprising a dataset, solvers to process the inputs, and a scorer to assess the outputs.
- **Solver**: A function that produces responses from a model based on the input and may include mechanisms for tool usage, multi-turn interactions, or prompt engineering.
- **Scorer**: Evaluates the model's outputs against expected results using various methods, including similarity checks, model scoring, or custom logic.
- **Dataset**: Reloading mechanism that supp